# Part 2: A short data story / micro-project

### Exercise: 

Create a short data-story based on the work we've done in class so far. (This exercise is a kind of "micro version" of what we'll be doing in the final project). Follow the directions in the bulleted list below when you create your data-story.

* __Find your own story to tell in the work on analyzing the SF Crime Data that we've done so far__ (Week 1 - Week 6). The idea is to choose an insight about crime in SF (perhaps how something changed over the years) and communicate that insight to a general audience reader.
* The web-page should be hosted on your GitHub Pages site (Week 7).
* The format must be the classic Magazine Genre presented on a single web-page (see the Segel & Heer paper for details).
* The story should have a brief introduction to the dataset so new readers from outside the class can understand what's going on.
* Your story should include three visualizations. Not more than three, not less than three (but multi-panel figures are OK). The figures should be one of each of the following types
    * One time-series / bar chart (it's OK to use the "fancy" plot-typs like calendar plots or polar bar-charts from Week 2, Part 4).
    * One map (use techniques from Week 3 and 4)
    * One interactive visualization in Bokeh (Week 6)

* __At a minimum, the Bokeh visualization should contain different data__ than the exercise we did for Week 6 (it's a plus if it's a new type of viz altogether).
* The two other visualization may be repetitions of figures created during the previous lectures, or they may be new.
* Make the figures nice. Specifically:
    * Aim to make the figures visually consistent (color, fonts, etc)
    * Follow the recommendations from my video on nice figures (Week 2, part 3)
* In terms of the amount of text, I envision something like 500-1500 words (including figure captions). Try to write in your own words - the LLMs have a tendency to write a lot of text and not be so precise. So if the writing is all elegantly written but empty prose, we will be critical. It is OK, however, to have the LLM help you get the grammer, etc. right.
* It is a plus if you can back up some of your findings with external sources, such as news stories from the area, looking up which building is located at some set of lat,lon coordinates, or similar. (So when you see something happening at some time/place in the data, see if you can understand it more deeply by investigating outside the dataset.) Use real references at the end of the text to organize your links to the outside world.

In [2]:
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
## Load files

file = "..\..\Data\Police_Department_Incident_Reports__Historical_2003_to_May_2018_20240130.csv"

df = pd.read_csv(file)
df = df[df['Date'].str.contains('2018') == False]

focuscrimes = set(['PROSTITUTION'])
focusdf = df[df['Category'].isin(focuscrimes)]

_One time-series / bar chart (it's OK to use the "fancy" plot-typs like calendar plots or polar bar-charts from Week 2, Part 4)._


In [ ]:
# fig, axes = plt.subplots(7, 2, figsize=(20, 40))
# fig.subplots_adjust(hspace=0.5)  # Adjust the vertical spacing between subplots

# for i, crime in enumerate(focuscrimes):
#     # Get the row and column index of the current crime
#     row = i // 2
#     col = i % 2
#     # Get the data for the current crime
#     years = focusdf[focusdf['Category'] == crime]['Date'].str.split('/').str[2].value_counts().sort_index()
#     # Plot the data
#     axes[row, col].bar(years.index, years)
#     axes[row, col].set_title(crime)
#     axes[row, col].set_xticklabels(years.index, rotation=90)
#     axes[row, col].set_ylabel('Occurences')
#     if row == 6:
#         axes[row, col].set_xlabel('Y e a r')


# plt.show()

_One map (use techniques from Week 3 and 4)_


_One interactive visualization in Bokeh (Week 6)_